# 01. Validação cruzada e aleatoriedade inicial 

In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
# Separando os dados em treinamento e teste:

x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [4]:
# Antes de rodar o classificador faremos um dummy classifier para ter uma acurácia base.

from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy='stratified')
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 50.96%


In [5]:

from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [6]:
# Fazendo a mudança do SEED e testando novamente:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia foi 76.84%


Documentação do [Cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html).

In [7]:
from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation 3: [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Accuracy com cross validation 3: [74.99, 76.57]


In [8]:
# Mudando o CV - cross-validation generator

from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation 3: [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Accuracy com cross validation 3: [74.24, 77.32]


Apesar de não estarmos sujeitos a aleatoriedade do SEED ainda estamos sujeitos ao parâmetro CV

Como escolher qual o melhor cv?

Dá uma olhada em [documentação externa](https://scikit-learn.org/stable/modules/cross_validation.html).

# 02. KFold e aleatoriedade.

## Aleatoriedade no cross_validate

In [9]:
def imprime_resultado(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy média: %.2f" % (media * 100))
    print("Accuracy intevalo: [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))
    

In [10]:
from sklearn.model_selection import cross_validate, KFold

SEED = 10
np.random.seed(SEED)

cv = KFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv)
imprime_resultado(results)

Accuracy média: 75.78
Accuracy intevalo: [74.37, 77.19]


In [11]:
# Rodando agora com o shuffle:

from sklearn.model_selection import cross_validate, KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv)
imprime_resultado(results)


Accuracy média: 75.76
Accuracy intevalo: [73.26, 78.26]


# 03. Estratificação:

## Simular situação horrível de azar.

Pode ser azar, como também pode ser uma proporção de exemplos desbalanceados entre as classes.


In [12]:
dados_azar = dados.sort_values(by=["vendido"], ascending=True)

x_azar = dados_azar[["preco", "idade_do_modelo","km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [13]:
# Rodando agora sem o shuffle:

from sklearn.model_selection import cross_validate, KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv)
imprime_resultado(results)

Accuracy média: 57.84
Accuracy intevalo: [34.29, 81.39]


In [14]:
# Rodando agora com o shuffle:

from sklearn.model_selection import cross_validate, KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv)
imprime_resultado(results)

Accuracy média: 75.78
Accuracy intevalo: [72.30, 79.26]


Documentação [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html).

In [15]:
# Como queremos manter a proporção entre as classes, então usaremos o StratifiedKFold

from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv)
imprime_resultado(results)

Accuracy média: 75.78
Accuracy intevalo: [73.55, 78.01]


In [16]:
cv.get_n_splits(x_azar, y_azar)

for train_index, test_index in cv.split(x_azar, y_azar):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = x_azar[train_index], x_azar[test_index]
     y_train, y_test = y_azar[train_index], y_azar[test_index]

TRAIN: [   0    1    2 ... 9997 9998 9999] TEST: [   3    9   11   17   28   30   54   61   72   89  110  117  131  147
  148  161  167  185  192  204  208  223  224  252  254  260  283  292
  306  311  313  316  338  353  365  383  385  392  393  410  422  423
  426  441  442  464  484  487  493  515  530  543  573  576  581  635
  670  695  709  710  714  716  723  735  739  741  746  748  752  758
  790  819  825  830  831  851  857  859  861  866  884  907  917  918
  948  950  951  972  984  988  995 1001 1028 1037 1052 1099 1101 1107
 1109 1110 1114 1118 1138 1142 1153 1157 1166 1167 1179 1205 1211 1218
 1239 1244 1258 1270 1284 1294 1309 1315 1325 1339 1345 1372 1374 1385
 1391 1392 1407 1408 1413 1417 1447 1461 1463 1470 1472 1484 1487 1492
 1500 1501 1520 1525 1541 1542 1555 1569 1580 1584 1588 1619 1625 1639
 1652 1656 1677 1693 1711 1729 1738 1740 1743 1760 1764 1766 1771 1796
 1810 1811 1821 1831 1844 1849 1854 1894 1899 1900 1921 1932 1944 1949
 1959 1960 1967 1984 1985 19

KeyError: "None of [Int64Index([   0,    1,    2,    4,    5,    6,    7,    8,   10,   12,\n            ...\n            9990, 9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999],\n           dtype='int64', length=9000)] are in the [columns]"

In [ ]:
list(cv.split(x_azar, y_azar))

[(array([   0,    1,    2, ..., 9997, 9998, 9999]),
  array([  25,   26,   51,   57,   59,   82,   88,   89,   93,   95,  102,
          109,  122,  123,  138,  139,  150,  167,  181,  190,  191,  202,
          215,  228,  231,  233,  236,  254,  255,  263,  267,  271,  300,
          313,  315,  327,  351,  362,  370,  373,  384,  393,  412,  413,
          431,  444,  452,  453,  460,  461,  471,  494,  501,  516,  524,
          540,  544,  552,  555,  560,  591,  593,  596,  613,  617,  622,
          629,  643,  656,  663,  678,  679,  700,  704,  722,  755,  767,
          768,  770,  796,  816,  818,  824,  831,  837,  842,  854,  867,
          884,  896,  903,  906,  912,  913,  935,  938,  950,  968,  973,
          978,  986,  987,  988, 1012, 1013, 1014, 1017, 1018, 1026, 1033,
         1038, 1047, 1060, 1064, 1066, 1067, 1068, 1071, 1072, 1073, 1075,
         1083, 1088, 1094, 1099, 1109, 1110, 1121, 1122, 1137, 1142, 1143,
         1145, 1167, 1172, 1174, 1199, 1229, 124

Obs: Verificar pq quando adiciona o shuffle o intervalo está dando diferente, mesmo setando o seed.

nessa [aula](https://cursos.alura.com.br/course/machine-learning-validando-modelos/task/48276) - 5:29

Embora o resultado seja mais fechado, é o mais recomendado. Principalmente quando há um desbalanço entre duas ou mais classes, é interessante utilizar o StratifiedKFold.

Reparem como ficariam os resultados se o deixassemos no aleatório. Inclusive, mesmo sem o shuffle, ele recupera bem a capacidade de prever o quão bom seria o treinamento. Vamos deletar shuffle

Paper: [Model selection: choosing estimators and their parameters](https://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html)

[Exercício](https://scikit-learn.org/stable/auto_examples/exercises/plot_cv_diabetes.html#sphx-glr-auto-examples-exercises-plot-cv-diabetes-py)

# 04. Dados Agrupáveis:

Documentação [randint](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html).

Gerando dados aleatórios de modelo de carro para simulação de agrupamento ao usar nosso estimador

In [ ]:
# Adicionando uma nova coluna:

np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()


,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


estando validação cruzada com GroupKFold:

In [2]:
# Entendendo o GroupKFold

from sklearn.model_selection import GroupKFold

X = [0.1, 0.2, 2.2, 2.4, 2.3, 4.55, 5.8, 8.8, 9, 10]
y = ["a", "b", "b", "b", "c", "c", "c", "d", "d", "d"]
groups = [1, 2, 1, 2, 2, 2, 3, 3, 3, 3]

gkf = GroupKFold(n_splits=3)
for train, test in gkf.split(X, y, groups=groups):
    print("Train = %s, Test = %s" % (train, test))


Train = [0 1 2 3 4 5], Test = [6 7 8 9]
Train = [0 2 6 7 8 9], Test = [1 3 4 5]
Train = [1 3 4 5 6 7 8 9], Test = [0 2]


In [ ]:
# Testando agora nosso estimador.

from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups=dados.modelo)
imprime_resultado(results)

Accuracy média: 75.78
Accuracy intevalo: [73.67, 77.90]


# 05. Pipeline de treino e validação:

![](https://iaml.it/blog/optimizing-sklearn-pipelines/images/pipeline-diagram.png)

## Cross Validation com [standart Scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC(gamma='auto')
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A Acurácia foi de %.2f %%" % acuracia)

A Acurácia foi de 77.48 %


In [ ]:


from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC(gamma='auto')
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultado(results)

Accuracy média: 58.00
Accuracy intevalo: [56.10, 59.89]


In [ ]:
scaler = StandardScaler()
scaler.fit(x_azar)
x_azar_escalado = scaler.transform(x_azar)

In [ ]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC(gamma='auto')
results = cross_validate(modelo, x_azar_escalado, y_azar, cv = cv, groups = dados.modelo,return_train_score=False)
imprime_resultado(results)

Accuracy média: 76.71
Accuracy intevalo: [74.30, 79.12]


Entretanto, no pré processamento de dados, quando escalamos as features, fit() é aplicado somente nos dados de treino, e aqui o fit() acontece em treino e teste. Até porque faremos dez separações baseadas nos grupos, para treinar e testar. Ou seja, não faremos o fit() somente uma vez. Devemos fazer esse fit() diversas vezes. Na verdade, deveríamos intercalar, fazendo uma vez esse fit, uma vez a validação, uma vez o fit, outra vez a validação, com outro conjunto de dados.

Toda vez que rodamos o GroupKFold, temos que fazer o fit() com uma parte de treino e rodar a validação com o teste. Se tivermos outro treino, rodaremos fit() para esse outro treino, e o teste com o resto. Assim, teremos um processo de duas fases:

- Primeiro, o pré processamento que escala;
- depois, a validação.

Esse processo deve ser rodado várias vezes, de acordo com os nossos grupos e com nossos com nossos splits. Então, precisamos pensar em outra forma, além de fazer o fit() somente uma vez e, depois, rodar o GroupKFold. Provavelmente, a estimativa obtida por esse processo está muito otimista.

Sendo assim, no Sklearn, criaremos um processo, uma sequência de passos, chamada [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), que está em um módulo separado. Começaremos importando-o com:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC(gamma='auto')

pipeline = Pipeline([('transformação', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo,return_train_score=False)
imprime_resultado(results)


Accuracy média: 76.68
Accuracy intevalo: [74.28, 79.08]


**Verificar isso de ter que rodar diversas vezes depois, pq n entendi direito**

Agora que temos diversas abordagens de validação você vai escolher a que acredita ser ideal para as características dos seus dados, do seu dataset, de acordo com o que deseja predizer.

A média e o intervalo providos pela validação cruzada diz o quão bem você espera que o modelo se comporte com dados previamente desconhecidos mas... se você usou cross validation com 10 folds, qual dos 10 modelos treinados você vai usar agora no mundo real?

A ideia é que a validação cruzada num conjunto de dados somente te diz o que você pode esperar. Ela não treina o seu modelo final.

Agora você tem o algoritmo do estimador (por exemplo um `DecisionTreeClassifier`) e pretende usá-lo no mundo real:

```
modelo = DecisionTreeClassifier(max_depth = n)
resultados = cross_validate(modelo, x, y, cv = ...)
```

Resultados indica o que esperar. [Para obter o modelo final](https://stats.stackexchange.com/questions/331250/how-to-train-the-final-model-after-cross-validation) basta treiná-lo em *todos os dados que você usou para o crossvalidation*:

```
modelo.fit(x, y)
```

Pronto, seu modelo está pronto para ser usado com o `predict` para classificar etc dados novos.